# Combine Processed Data

In [ ]:
import pandas as pd

import sys
import os
from pathlib import Path
from config import Config as paths
project_root = Path("..").resolve()
sys.path.insert(0, str(project_root))
from data_cleaning.cleaners.baseCleaner import BaseCleaner
base_cleaner = BaseCleaner()


## Load Data

In [ ]:
microbiology = pd.read_parquet(f"{paths.STORE_MICROBIOLOGY_PATH}/microbiology_dedub.parquet")
ceiling_data = pd.read_parquet(f"{paths.STORE_CEILING_DATA_PATH}/ceiling_data_summary.parquet")
diagnosis = pd.read_parquet(f"{paths.STORE_DIAGNOSIS_DATA_PATH}/diagnosis_combined_summary.parquet")
dialysis = pd.read_parquet(f"{paths.STORE_DIALYSIS_DATA_PATH}/dialysis_episode_summary.parquet")
medicine = pd.read_parquet(f"{paths.STORE_MEDICINE_DATA_PATH}/medicine_episode_summary.parquet")
orbit = pd.read_parquet(f"{paths.STORE_ORBIT_DATA_PATH}/orbit_surgery_combined.parquet")
outcome = pd.read_parquet(f"{paths.STORE_OUTCOME_DATA_PATH}/outcome_data.parquet")
lab_vitals = pd.read_parquet(paths.STORE_PATH + "/laboratory_vitals" + "/laboratory_vitals_first_measurements_24_hours.parquet")


In [ ]:
cols_at_beginning = ['episode_id','patient_id','sample_id']
microbiology = microbiology[cols_at_beginning + [col for col in microbiology.columns if col not in cols_at_beginning]]

## Merge Data

In [ ]:
final_df = microbiology.merge(ceiling_data, on='episode_id', how='left')
final_df = final_df.merge(diagnosis, on='episode_id', how='left')
final_df = final_df.merge(dialysis, on='episode_id', how='left')
final_df = final_df.merge(medicine, on='episode_id', how='left')
final_df = final_df.merge(orbit, on='episode_id', how='left')
final_df = final_df.merge(outcome, on='episode_id', how='left')
final_df = final_df.merge(lab_vitals, on='episode_id', how='left')
final_df = base_cleaner.clean_data(final_df)


## Save Combined Data

In [ ]:
if not os.path.exists(paths.STORE_PATH):
    os.makedirs(paths.STORE_PATH)

final_df.to_parquet(f"{paths.STORE_PATH}/combined_data.parquet")